In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [10]:
# Code Generation
message = "1101"  # 4-bit binary message

nr_codewords = int(100)
bits_info = torch.randint(2, (nr_codewords, 4), dtype=torch.float)

print(bits_info)

tensor([[1., 0., 1., 0.],
        [1., 1., 1., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 1., 0., 1.],
        [1., 0., 1., 1.],
        [1., 1., 1., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 1.],
        [0., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 1.],
        [1., 1., 1., 0.],
        [0., 1., 1., 0.],
        [1., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 0.],
        [1., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 1., 1.],
        [1., 1., 0., 1.],
        [1., 1., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0.,

Hamming(7,4) Encoder

In [11]:
class hamming_encode(torch.nn.Module):
    def __init__(self):
        super(hamming_encode, self).__init__()

        # Define the generator matrix for Hamming(7,4)
        self.generator_matrix = torch.tensor([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [1, 1, 0, 1],
            [1, 0, 1, 1],
            [0, 1, 1, 1],
        ], dtype=torch.float)

    def forward(self, input_data):
        # Ensure input_data has shape (batch_size, 4)
        assert input_data.size(1) == 4, "Input data must have 4 bits."

        # Perform matrix multiplication to encode the data
        encoded_data = torch.matmul(input_data, self.generator_matrix.t()) % 2

        return encoded_data

In [20]:
encoder = hamming_encode()
encoded_codeword = encoder(bits_info)
print(encoded_codeword)

tensor([[1., 0., 1., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 0., 0., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 1., 0., 0.],
        [1., 0., 1., 1., 0., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1.],
        [1., 0., 0., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1.],
        [1., 1., 0., 1., 1., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0.],
        [0., 1., 1., 0., 1., 1., 0.],
        [1., 1., 0., 0., 0., 1., 1.],
        [0., 0., 1., 0., 0., 1., 1.],
        [0., 1., 0., 1., 0., 1., 0.],
        [0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 0., 0.],
        [0., 1., 1., 1., 0., 0., 1.],
        [0., 1., 1., 0., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 0., 0., 1.],
        [0., 1., 0., 0., 1., 0., 1.],
        [1.,

BPSK Modulator + Noise

In [27]:
def bpsk_modulator(data, symbol_rate, carrier_freq, snr_dB):
# data = encoded_codeword  # Binary data
# symbol_rate = 15  # Symbol rate in Hz
# carrier_freq = 100  # Carrier frequency in Hz
# snr_dB = 15  # Signal-to-noise ratio in dB

    for i in range(data.shape[0]):
        bits = data[i]
        bits = 2 * bits - 1
        # Time vector
        time = torch.arange(0, len(bits) / symbol_rate, 1 / symbol_rate)
    
        # Generate carrier signal
        carrier = torch.cos(2 * torch.pi * carrier_freq * time)
    
        # Modulate the signal
        modulated_signal = bits * carrier
    
        # Add Gaussian noise to the signal
        noise_power = torch.tensor(10**(-snr_dB / 10))
        noise = torch.sqrt(noise_power) * torch.randn(len(modulated_signal))
        noised_signal = modulated_signal + noise
        data[i] = noised_signal

   
        return time, data

In [29]:
data = encoded_codeword  # Binary data
symbol_rate = 15  # Symbol rate in Hz
carrier_freq = 100  # Carrier frequency in Hz
snr_dB = 15  # Signal-to-noise ratio in dB

# Modulate the signal
time, modulated_noise_signal = bpsk_modulator(data, symbol_rate, carrier_freq, snr_dB)
print(modulated_noise_signal)

tensor([[-1.6549e+01,  6.4436e-01,  9.8624e-01, -2.4576e+02,  1.3624e+00,
         -5.8915e-01,  3.9532e+01],
        [ 2.3286e+00, -3.1254e-01,  1.8155e-01,  4.5813e+00, -1.4976e-01,
         -1.0768e+00, -2.7593e+00],
        [-2.9799e+01,  1.1822e+00, -5.6034e-01, -3.2579e+01,  8.9268e-01,
         -7.0768e-01, -7.2879e-01],
        [-3.0981e+01,  1.4730e-01,  5.5268e-01,  5.9568e+00, -1.8666e-01,
         -8.5399e-01,  4.4770e+00],
        [-3.6226e+01,  1.5054e+00, -1.1986e+00, -3.2184e+01,  2.1577e-01,
         -6.5021e-01, -1.1434e+00],
        [-3.5461e+00, -4.5158e-01,  7.2040e-01, -7.8059e-01, -7.7504e-01,
          8.9410e-01, -3.4493e+01],
        [ 1.5931e+00,  5.1758e-01, -3.1251e-01,  3.8725e+00,  8.7301e-02,
         -9.2234e-01, -3.7106e+01],
        [ 3.7625e+00, -5.6812e-01, -7.7691e-01,  3.6749e+00, -2.6043e-01,
         -2.4156e-01,  1.9259e+00],
        [-3.5151e+01,  5.6294e-01, -4.2629e-01,  2.4347e+00,  3.1654e-01,
         -1.3424e+00, -2.9454e+00],
        [ 

LLR Log-likelihood

In [147]:
import numpy as np

def ldpc_decode(received_codeword, max_iterations=50):
    # LDPC Parity-check matrix (7,4) example
    H = np.array([
        [1, 0, 1, 1, 0, 0, 0],
        [0, 1, 0, 0, 1, 0, 1],
        [0, 0, 0, 1, 0, 1, 0]
    ])

    # LDPC Generator matrix (7,4) example
    G = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 1, 1],
        [1, 0, 1, 1],
        [1, 1, 0, 1]
    ])

    # Initialize the received codeword and the decoded message
    y = np.array(list(map(int, received_codeword)))
    x_hat = np.zeros(G.shape[1])

    for iteration in range(max_iterations):
        # Syndrome check
        syndrome = np.mod(np.dot(H, y), 2)

        if np.all(syndrome == 0):
            # If the syndrome is all zeros, the decoding is successful
            break

        # Compute the log-likelihood ratios (LLRs) using the sum-product algorithm
        llrs = np.zeros(G.shape[1])
        for j in range(G.shape[1]):
            connected_checks = np.nonzero(H[:, j])[0]
            llrs[j] = 2 * y[j] / np.tanh(np.sum(np.arctanh(np.tanh(y[i] / 2)) for i in connected_checks))

        # Make hard decisions based on LLRs
        x_hat = np.where(llrs > 0, 1, 0)

        # Update the received codeword
        y = np.mod(np.dot(G, x_hat) + y, 2)

    decoded_message = ''.join(map(str, x_hat.astype(int)))
    return decoded_message

def main():
    # Example usage
    received_codeword = "1101001"  # Received codeword with possible errors
    decoded_message = ldpc_decode(received_codeword)

    print(f"Received codeword: {received_codeword}")
    print(f"Decoded message: {decoded_message}")

if __name__ == "__main__":
    main()
